# CCN Assignment 6: Generative	Adversarial	Networks

In [1]:
%matplotlib inline

import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.dataset import concat_examples
from chainer.datasets import TupleDataset
from chainer.functions.evaluation import accuracy
from chainer.functions.loss import softmax_cross_entropy
from chainer import link
from chainer import reporter

import matplotlib.pyplot as plt
from utils import get_mnist

In [2]:
# make loss function such that loss is high when discriminator 
# classifies generator network output as fake 0, and
# low when generator network output as being a real 0

class Generator(Chain):
    def __init__(self, n_units, dim_output):
        super(Generator, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(n_units, n_units) # fully connected layer
            self.batch_norm = L.BatchNormalization(n_units) # batch normalization
            self.deconv = L.Deconvolution2D(None, (dim_output, dim_output)) # deconvolution layer
            
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = self.batch_norm(h1)
        h3 = self.deconv(h2)
        y = F.sigmoid(h3)
        return y

In [3]:
class Discriminator(Chain):
    def __init__(self, n_out):
        super(Discriminator, self).__init__()
        with self.init_scope():
            self.conv = L.Convolution2D(None, out_channels=5, ksize=5, stride=1, pad=0)
            self.fc1 = L.Linear(None, n_out)
            
        def __call__(self, x):
            h1 = self.conv1(x)
            h2 = F.max_pooling_2d(h1, ksize=5, stride=1, pad=0) # max pooling layer
            y = F.relu(self.fc1(h2))
            return y

In [ ]:
def run(generator, discriminator, classifier, optimizer, true_train_iter, true_val_iter, fake_train_iter):
    classifier.cleargrads()
    
#     First generate a sample with the generator network.
#     Classify it with the discriminator network, 
#     calculate the loss such that you enhance samples that the discriminator thinks are real 
#     and update the networks. 
#     Next calculate the loss of the generated sample enhancing those samples 
#     that the discriminator correctly recognizes as fake. 
#     Combine this with the loss that the discriminator gets on real images 
#     and update the networks based on this combined loss.
    
    # get next mini-batch
    batch = 

In [6]:
def main():
    # generate the real data
    train_data, test_data = get_mnist(n_train=1000, n_test=100, with_label=False, classes = [0])
    
    # generate the fake data
    fake_train = np.random.rand(1000,28,28)
#     fake_test = np.random.rand(100,28,28)
    
    # initialize interators
    true_train_iter = iterators.SerialIterator(train_data, batch_size=64, shuffle=True)
    true_val_iter = iterators.SerialIterator(test_data, batch_size=100, repeat=False, shuffle=False)
    fake_train_iter = iterators.SerialIterator(fake_train, batch_size=64, shuffle=True)
    
    # define models
    generator = Generator(784, 28)
    discriminator = Discriminator(1)
    classifier = L.Classifier(discriminator, F.sigmoid_cross_entropy)
    optimizer = optimizers.SGD()
    optimizer.setup(classifier)
    
if __name__ == "__main__":
    main()